The cells are copied code from [one dev webpage](https://dev.to/zenulabidin/python-asyncio-great-for-i-o-do-you-use-it-for-other-things-10ei)

I was unable to fully understand the content, but it seemed to be somewhat valuable examples.

In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
import asyncio
import datetime

async def display_date():
    loop = asyncio.get_running_loop()
    # You cancel the loop with loop.stop()
    end_time = loop.time() + 5.0
    while True:
        print(datetime.datetime.now())
        if (loop.time() + 1.0) >= end_time:
            break
        await asyncio.sleep(1)

asyncio.run(display_date())
# Outputs the current date every second for five seconds.

2020-10-09 16:40:42.944362
2020-10-09 16:40:43.946016
2020-10-09 16:40:44.947776
2020-10-09 16:40:45.949693
2020-10-09 16:40:46.951384


In [3]:
import asyncio

async def factorial(name, number):
    f = 1
    for i in range(2, number + 1):
        print(f"Task {name}: Compute factorial({i})...")
        await asyncio.sleep(1)
        f *= i
    print(f"Task {name}: factorial({number}) = {f}")

async def main():
    # Schedule three calls *concurrently*. If you selectively
    # cancel one of these function calls, the entire gather()
    # call gets cancelled.
    await asyncio.gather(
        factorial("A", 2),
        factorial("B", 3),
        factorial("C", 4),
    )

asyncio.run(main())

#Task A: Compute factorial(2)...
#Task B: Compute factorial(2)...
#Task C: Compute factorial(2)...
#<one second later...>
#Task A: factorial(2) = 2
#Task B: Compute factorial(3)...
#Task C: Compute factorial(3)...
#<one second later...>
#Task B: factorial(3) = 6
#Task C: Compute factorial(4)...
#<one second later...>
#Task C: factorial(4) = 24

Task A: Compute factorial(2)...
Task B: Compute factorial(2)...
Task C: Compute factorial(2)...
Task A: factorial(2) = 2
Task B: Compute factorial(3)...
Task C: Compute factorial(3)...
Task B: factorial(3) = 6
Task C: Compute factorial(4)...
Task C: factorial(4) = 24


In [4]:
import asyncio

async def readbytes(name, number):
    with open(name, "rb") as f:
        s = f.read(number)
        await asyncio.sleep(number)
        print(f"File contents: {s}")

async def writebytes(name, number, string):
    with open(name, "w") as f:
        f.write(string)
        await asyncio.sleep(number)
        print(f"Wrote: {string}")

async def baz():
    await asyncio.gather(
        readbytes("/dev/urandom", 2),
        readbytes("/dev/random", 3),
        readbytes("/dev/zero", 4),
        writebytes("/tmp/spam", 2, "spamspam"),
        writebytes("/tmp/eggs", 3, "eggseggs"),
        writebytes("/tmp/bar", 4, "barbar"),
    )

%time asyncio.run(baz())

File contents: b'\x9dP'
Wrote: spamspam
File contents: b'\x03\x06\x1e'
Wrote: eggseggs
File contents: b'\x00\x00\x00\x00'
Wrote: barbar
CPU times: user 10.7 ms, sys: 4.07 ms, total: 14.8 ms
Wall time: 4.02 s


Let's change one of the task to `5` seconds.

In [5]:
import asyncio

async def readbytes(name, number):
    with open(name, "rb") as f:
        s = f.read(number)
        await asyncio.sleep(number)
        print(f"File contents: {s}")

async def writebytes(name, number, string):
    with open(name, "w") as f:
        f.write(string)
        await asyncio.sleep(number)
        print(f"Wrote: {string}")

async def baz():
    await asyncio.gather(
        readbytes("/dev/urandom", 5),
        readbytes("/dev/random", 3),
        readbytes("/dev/zero", 4),
        writebytes("/tmp/spam", 2, "spamspam"),
        writebytes("/tmp/eggs", 3, "eggseggs"),
        writebytes("/tmp/bar", 4, "barbar"),
    )

%time asyncio.run(baz())

Wrote: spamspam
File contents: b'\xa3\xdc\x0c'
Wrote: eggseggs
File contents: b'\x00\x00\x00\x00'
Wrote: barbar
File contents: b'\x8d\xc6ii\x8d'
CPU times: user 10.2 ms, sys: 6.85 ms, total: 17.1 ms
Wall time: 5 s


**Again** with a diff task

In [6]:
import asyncio

async def readbytes(name, number):
    with open(name, "rb") as f:
        s = f.read(number)
        await asyncio.sleep(number)
        print(f"File contents: {s}")

async def writebytes(name, number, string):
    with open(name, "w") as f:
        f.write(string)
        await asyncio.sleep(number)
        print(f"Wrote: {string}")

async def baz():
    await asyncio.gather(
        readbytes("/dev/urandom", 1),
        readbytes("/dev/random", 3),
        readbytes("/dev/zero", 4),
        writebytes("/tmp/spam", 2, "spamspam"),
        writebytes("/tmp/eggs", 5, "eggseggs"),
        writebytes("/tmp/bar", 4, "barbar"),
    )

%time asyncio.run(baz())

File contents: b'\x17'
Wrote: spamspam
File contents: b'E\x89?'
File contents: b'\x00\x00\x00\x00'
Wrote: barbar
Wrote: eggseggs
CPU times: user 14.1 ms, sys: 2.83 ms, total: 17 ms
Wall time: 5 s


Next cell **should** be **modified** because when run in jupyter, it'll kill the jupyter kernel, which seems to use `asyncio`.